# Module 12 Challenge
## Deliverable 2: Scrape and Analyze Mars Weather Data

In [1]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup as soup
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
browser = Browser('chrome')

### Step 1: Visit the Website

Use automated browsing to visit the [Mars Temperature Data Site](https://static.bc-edx.com/data/web/mars_facts/temperature.html). Inspect the page to identify which elements to scrape.

   > **Hint** To identify which elements to scrape, you might want to inspect the page by using Chrome DevTools to discover whether the table contains usable classes.


In [ ]:
# Visit the website
# https://static.bc-edx.com/data/web/mars_facts/temperature.html
url = "https://static.bc-edx.com/data/web/mars_facts/temperature.html"
browser.visit(url)

### Step 2: Scrape the Table

Create a Beautiful Soup object and use it to scrape the data in the HTML table.

Note that this can also be achieved by using the Pandas `read_html` function. However, use Beautiful Soup here to continue sharpening your web scraping skills.

In [ ]:
# Create a Beautiful Soup Object
html = browser.html

soup = soup(html, 'html.parser')

In [ ]:
# Extract all rows of data
data= soup.find('table')

data_rows =data.tbody.find_all("tr", class_="data-row")

### Step 3: Store the Data

Assemble the scraped data into a Pandas DataFrame. The columns should have the same headings as the table on the website. Here’s an explanation of the column headings:

* `id`: the identification number of a single transmission from the Curiosity rover
* `terrestrial_date`: the date on Earth
* `sol`: the number of elapsed sols (Martian days) since Curiosity landed on Mars
* `ls`: the solar longitude
* `month`: the Martian month
* `min_temp`: the minimum temperature, in Celsius, of a single Martian day (sol)
* `pressure`: The atmospheric pressure at Curiosity's location

In [ ]:
# Create an empty list
row_list = []

# Loop through the scraped data to create a list of rows
for row in data_rows:
    row_list.append([n.text.strip() for n in row.find_all('td')])

In [ ]:
# Create a Pandas DataFrame by using the list of rows and a list of the column names

#Build list for column namers
column_list = ['id', 'terrestrial_date', 'sol', 'ls', 'month', 'min_temp', 'pressure']
#Assemble df from column and row lists
mars_df = pd.DataFrame(row_list, columns=column_list)


In [ ]:
# Confirm DataFrame was created successfully
mars_df.head()

### Step 4: Prepare Data for Analysis

Examine the data types that are currently associated with each column. If necessary, cast (or convert) the data to the appropriate `datetime`, `int`, or `float` data types.

  > **Hint** You can use the Pandas `astype` and `to_datetime` methods to accomplish this task.


In [ ]:
# Examine data type of each column
mars_df.dtypes

In [ ]:
# Change data types for data analysis

# Convert 'terrestrial date to datetime
mars_df['terrestrial_date'] = pd.to_datetime(mars_df['terrestrial_date'])
# Convert other objexts to string and float
mars_df = mars_df.astype({'sol':'int', 'ls':'int', 'month':'int', \
                          'min_temp':'float', 'pressure':'float'})


In [ ]:
# Confirm type changes were successful by examining data types again
mars_df.dtypes

### Step 5: Analyze the Data

Analyze your dataset by using Pandas functions to answer the following questions:

1. How many months exist on Mars?
2. How many Martian (and not Earth) days worth of data exist in the scraped dataset?
3. What are the coldest and the warmest months on Mars (at the location of Curiosity)? To answer this question:
    * Find the average the minimum daily temperature for all of the months.
    * Plot the results as a bar chart.
4. Which months have the lowest and the highest atmospheric pressure on Mars? To answer this question:
    * Find the average the daily atmospheric pressure of all the months.
    * Plot the results as a bar chart.
5. About how many terrestrial (Earth) days exist in a Martian year? To answer this question:
    * Consider how many days elapse on Earth in the time that Mars circles the Sun once.
    * Visually estimate the result by plotting the daily minimum temperature.


In [ ]:
# 1. How many months are there on Mars?

# Count instances of each month in month column
months = mars_df['month'].value_counts()
# Convert indext to int for correct sorting
months.index = months.index.astype(int)
# Sort index ascending by month
months.sort_index()

In [ ]:
# 2. How many Martian days' worth of data are there?

# Count index for total days
martian_days = len(mars_df.index)
martian_days

In [ ]:
# 3. What is the average low temperature by month?

# Copy df for reformatting
mars_pos =mars_df.copy()

# Convert 'min-temp' to string to strip negative values 
mars_pos['min_temp'] = mars_pos['min_temp'].astype(str)
# Strip negative sign from 'min_temp' and convert back to float
mars_pos['min_temp'] = mars_pos['min_temp'].str.replace('-','').astype(float)
# Find the mean of each month
ave_low = mars_pos.groupby(['month'])['min_temp'].mean()
# Set index to int to properly sort by month
ave_low.index = ave_low.index.astype(int)
# Sort index by month
ave_low = ave_low.sort_index()
# Insert negative sign using -abs
ave_low = -abs(ave_low)

ave_low

In [ ]:
# Plot the average temperature by month

# Bar plot from series
plt.bar(ave_low.index, ave_low.values, width=.5) 
plt.xlabel("Month")
plt.ylabel("Temperature in Celsius")
plt.xticks(ave_low.index, rotation=90)

plt.show()

In [ ]:
# Identify the coldest and hottest months in Curiosity's location

#Coldest Average Temps by month
#Starting with mars_pos df, since it has stripped the negative from all values. Find mean, sort
ave_low_cold = mars_pos.groupby(['month'])['min_temp'].mean().sort_values(ascending=False)
ave_low_cold = -abs(ave_low_cold)
ave_low_cold
# Convert series to df for plotting
ave_low_df = pd.DataFrame(ave_low_cold)
# Create bar plot
ave_low_df.plot.bar(xlabel="Month", ylabel="Temperature in Celsius", legend=False)
plt.show()

In [ ]:
# 4. Average pressure by Martian month

# Copy main df for sorting
pressure_df= mars_df.copy()
# Convert pressure to float to allow for large calculation
pressure_df['pressure'] = pressure_df['pressure'].astype(float)
# Group average pressure by month
ave_pressure = pressure_df.groupby(['month'])['pressure'].mean()
# Set index to in for proper sorting
ave_pressure.index = ave_pressure.index.astype(int)
# Sort index
ave_pressure = ave_pressure.sort_index()

ave_pressure

In [ ]:
# Plot the average pressure by month

# Sort atmospheric pressure average by month
ave_pressure_sort = pressure_df.groupby(['month'])['pressure'].mean().sort_values()
ave_pressure_sort
# Convert series to df for plotting
ave_pressure_df = pd.DataFrame(ave_pressure_sort)
# Bar plot
ave_pressure_df.plot.bar(xlabel="Month", ylabel="Atmospheric Pressure", legend=False)
plt.show()

In [ ]:
# 5. How many terrestrial (earth) days are there in a Martian year?

# extract the min temp for every day in the positive formatted df
days = mars_pos['min_temp']
#correct min_temp to negative
days = -abs(days)
days

# Plot each day to infer Martian year based on peaks of hot and cold temps
plt.plot(days)
plt.xlabel("Number of Terrestrial Days")
plt.ylabel("Minimum Temperature")

On average, the third month has the coldest minimum temperature on Mars, and the eighth month is the warmest. But it is always very cold there in human terms!



Atmospheric pressure is, on average, lowest in the sixth month and highest in the ninth.

The distance from peak to peak is roughly 1425-750, or 675 days. A year on Mars appears to be about 675 days from the plot. Internet search confirms that a Mars year is equivalent to 687 earth days.

### Step 6: Save the Data

Export the DataFrame to a CSV file.

In [ ]:
# Write the data to a CSV
mars_df.to_csv('output_data/mars_data.csv')

In [ ]:
browser.quit()